<a href="https://colab.research.google.com/github/anna-alt/AI-Lab/blob/main/SceneRecognition1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, SubsetRandomSampler, ConcatDataset
from torchvision import datasets, transforms
from torchvision import models

import numpy as np
import sklearn



In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

device

device(type='cuda')

In [13]:
trainset_size = 25000
val_size = 5000
num_epochs = 10
image_size = (224,224)


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Resize(image_size),
                              
                              ])
# Download and load the training data
trainset = datasets.SUN397('/content/drive/MyDrive/Data', download= False, transform=transform)
splittrain = torch.utils.data.random_split(trainset, [trainset_size, len(trainset) - trainset_size])[0]
train_loader = DataLoader(splittrain, batch_size=32,num_workers = 2, shuffle=True)

validationset = datasets.SUN397("/content/drive/MyDrive/Data", download=False, transform=transform)
splitvalidation = torch.utils.data.random_split(validationset, [val_size, len(trainset) - val_size])[0]
val_loader = DataLoader(splitvalidation, batch_size=32,num_workers = 2, shuffle=True)



In [ ]:
for image, label in train_loader:
  labels = np.array(image)
  numpy_labels = label.numpy()
  print(labels)
  break


In [ ]:
model = models.resnet50(weights = "DEFAULT")
model.to(device)

In [8]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, betas=(0.9, 0.999), eps=1e-8)
criterion = nn.CrossEntropyLoss()


In [9]:
def train(epoch, log_interval=2000):
    # Set model to training mode
    model.train()
    
    # Loop over each batch from the training set
    for batch_idx, (data, target) in enumerate(train_loader):
        # Copy data to GPU if needed
        data = data.to(device)
        target = target.to(device)

        # Zero gradient buffers
        optimizer.zero_grad() 
        
        # Pass data through the network
        output = model(data)

        # Calculate loss
        loss = criterion(output, target)

        # Backpropagate
        loss.backward()
        
        # Update weights
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data.item()))

In [10]:

def validate(loss_vector, accuracy_vector):
    model.eval()
    val_loss, correct = 0, 0
    for data, target in val_loader:
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        val_loss += criterion(output, target).data.item()
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    val_loss /= len(val_loader)
    loss_vector.append(val_loss)

    print(pred)

    accuracy = 100. * correct.to(torch.float32) / len(val_loader.dataset)
    accuracy_vector.append(accuracy)

    
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        val_loss, correct, len(val_loader.dataset), accuracy))

In [11]:
def Predic(loss_vector,accuracy_vector):
    val_loss, correct = 0, 0
    for data, target in val_loader:
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        val_loss += criterion(output, target).data.item()
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()
    print( pred)
    print("/n")
    print(target)


In [ ]:
lossv, accv = [], []
for epoch in range(1, num_epochs + 1):
    train(epoch)
    validate(lossv, accv)
    Predic(lossv,accv)

Train Epoch: 1 [0/25000 (0%)]	Loss: 2.109844


/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1385478 bytes but only got 27197. Skipping tag 513
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 7276331008 bytes but only got 0. Skipping tag 7
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 653262848 bytes but only got 0. Skipping tag 4
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1873215488 bytes but only got 0. Skipping tag 4
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1073020928 bytes but only got 27197. Skipping tag 4
  warnings.warn(
/usr/local/lib/python3.8/dist-pack

tensor([ 37,  48,  48, 115, 213, 249,  10, 141], device='cuda:0')

Validation set: Average loss: 1.2628, Accuracy: 3345/5000 (67%)

tensor([150, 337, 111,  80, 279,  46, 193,  48], device='cuda:0')
/n
tensor([150, 337, 111, 279, 112,  46, 193,  48], device='cuda:0')
Train Epoch: 2 [0/25000 (0%)]	Loss: 0.612612


/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1385478 bytes but only got 27197. Skipping tag 513
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 7276331008 bytes but only got 0. Skipping tag 7
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 653262848 bytes but only got 0. Skipping tag 4
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1873215488 bytes but only got 0. Skipping tag 4
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1073020928 bytes but only got 27197. Skipping tag 4
  warnings.warn(
/usr/local/lib/python3.8/dist-pack

tensor([127,  25, 160,  48,  77, 179,  81, 214], device='cuda:0')

Validation set: Average loss: 1.0600, Accuracy: 3599/5000 (72%)

tensor([177, 370, 276, 167, 173,  27, 290, 279], device='cuda:0')
/n
tensor([177, 370, 270, 167, 264,  27, 290, 279], device='cuda:0')
Train Epoch: 3 [0/25000 (0%)]	Loss: 0.275911


...

tensor([352, 393, 318,  82, 193, 227, 110, 336], device='cuda:0')

Validation set: Average loss: 1.0880, Accuracy: 3615/5000 (72%)

tensor([ 55, 288,  82,  81,  95, 213, 104, 195], device='cuda:0')
/n
tensor([ 55, 288,   0,  81,  95, 213, 213, 150], device='cuda:0')
Train Epoch: 4 [0/25000 (0%)]	Loss: 0.264856


/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1385478 bytes but only got 27197. Skipping tag 513
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 7276331008 bytes but only got 0. Skipping tag 7
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 653262848 bytes but only got 0. Skipping tag 4
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1873215488 bytes but only got 0. Skipping tag 4
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1073020928 bytes but only got 27197. Skipping tag 4
  warnings.warn(
/usr/local/lib/python3.8/dist-pack

tensor([ 75, 185, 245, 108,  11, 146, 115,  56], device='cuda:0')

Validation set: Average loss: 1.1715, Accuracy: 3619/5000 (72%)

tensor([ 86,  23, 174, 190, 375,  82,   0, 341], device='cuda:0')
/n
tensor([ 86,  23, 174, 190, 375,  82,   0, 341], device='cuda:0')
Train Epoch: 5 [0/25000 (0%)]	Loss: 0.047276


/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1385478 bytes but only got 27197. Skipping tag 513
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 7276331008 bytes but only got 0. Skipping tag 7
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 653262848 bytes but only got 0. Skipping tag 4
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1873215488 bytes but only got 0. Skipping tag 4
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1073020928 bytes but only got 27197. Skipping tag 4
  warnings.warn(
/usr/local/lib/python3.8/dist-pack

tensor([ 97,   4, 161, 175,  47, 193, 262,  34], device='cuda:0')

Validation set: Average loss: 1.1506, Accuracy: 3630/5000 (73%)

tensor([253, 351, 174, 300, 308,  10,  63, 206], device='cuda:0')
/n
tensor([253, 351, 174, 300, 308,  10,  11, 207], device='cuda:0')
Train Epoch: 6 [0/25000 (0%)]	Loss: 0.070013


In [ ]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))